### Layered feedback control circuit 

 $P_1 + I_1 \quad [d_1]<->[a_1] \quad C_1, \qquad C_1 + I_1 \quad [d_2]<->[a_2] \quad C_2 \\
    C_2 + T_1 \quad [d_3]<->[a_3]\quad  A_T, \qquad R + A_T \quad -->[k_1]\quad  T_1 \\
    A_T  -->[\beta_A] \quad M_S + A_T, \qquad M_S -->[d_M] \quad  \varnothing\\
    M_S  -->[k_{tl}]\quad  M_S + S, \qquad S -->[d_S]\quad \varnothing  \\
    S  -->[k_r]\quad  S^*, \qquad S^* -->[d_{S^*}] \quad \varnothing \\
     S^* + I_2 \quad [d_4]<->[a_4]\quad  C_C, \qquad C_C + T_2 \quad [d_5]<->[a_5]\quad  A_C \\
    R + A_C \quad -->[k_2]\quad  T_2 \\
    A_C  -->[\beta_C]\quad R + M_G + A_C, \qquad R -->[d_R]\quad  \varnothing\\
    M_G  -->[k_{G}]\quad  M_G + G, \qquad G -->[d_G]\quad\varnothing \\
    G  -->[k_{r_2}] \quad G^*, \qquad G^* -->[d_{G^*}]\quad \varnothing $

In [1]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from auto_reduce import *
import numpy as np
from sympy import symbols

In [13]:
# Post conservation law and other approximations phenomenological model at the RNA level
n = 15 # Number of states : P, A_T, C1, T1, R, C2, M_S, S, Ss, C_C, A_C, M_G, G, Gs, T2
ninp = 2 # Number of inputs, I1, I2
nouts = 1 # Number of outputs, Gs

# Inputs by user 
x_init = np.zeros(n)
x_init[0] = 10
x_init[1] = 10000
x_init[4] = 10000
x_init[6] = 10000
C = np.zeros((nouts,n), dtype=int)
C[0][13] = 1 #Gs

error_tol = 30000
# System dynamics symbolically

P1 = symbols('P1')
A_T = symbols('A_T')
C1 = symbols('C1') # P:I1
T1 = symbols('T1')
R = symbols('R')
C2 = symbols('C2') # C1:I1
M_S = symbols('M_S')
S = symbols('S') 
Ss = symbols('Ss')

I2 = symbols('I2')
C_C = symbols('C_C')
A_C = symbols('A_C') 
M_G = symbols('M_G')
G = symbols('G')
Gs = symbols('Gs') 
T2 = symbols('T2')
I1 = symbols('I1')
# x = [x0, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13, x14, x15, x16]
x = [P1, A_T, C1, T1, R, C2, M_S, S, Ss, C_C, A_C, M_G, G, Gs, T2]
u = [I1, I2]

a1 = symbols('a1')
d1 = symbols('d1')
a2 = symbols('a2')
d2 = symbols('d2')
a3 = symbols('a3')
d3 = symbols('d3')
k1 = symbols('k1')
b_A = symbols('b_A')
d_M = symbols('d_M')
k_tl = symbols('k_tl')
d_S = symbols('d_S')
k_r = symbols('k_r')
d_Ss = symbols('d_Ss')
a4 = symbols('a4')
d4 = symbols('d4')
a5 = symbols('a5')
d5 = symbols('d5')
k2 = symbols('k2')
b_C = symbols('b_C')
d_R = symbols('d_R')
k_G = symbols('k_G')
d_G = symbols('d_G')
d_Gs = symbols('d_Gs')
k_r2 = symbols('k_r2')


T1_tot = symbols('T1_tot')
T2_tot = symbols('T2_tot')
# R_tot = symbols('R_tot')

params = [a1,d1,a2,d2,a3,d3,k1,b_A,d_M,k_tl,d_S,k_r,d_Ss,a4,d4,a5,d5,k2,b_C,d_R,k_G,d_G,d_Gs,k_r2,T1_tot,T2_tot]

# Number of states : 
# P (0), A_T (1), C1 (2), T1 (3), R (4), C2 (5), M_S (6), S (7), Ss (8),
# C_C (9), A_C (10), M_G (11), G (12), Gs (13), T2 (14)
# [P1, A_T, C1, T1, R, C2, M_S, S, Ss, C_C, A_C, M_G, G, Gs, T2]
f0 = d1*C1  #P1
g0 = - a1*P1*I1
f1 = a3*C2*T1 - d3*A_T - k1*R*A_T  #A_T
g1 = 0
f2 = d2*C2 - d1*C1  #C1
g2 = a1*P1*I1 - a2*C1*I1
f3 = d3*A_T - a3*C2*T1 + k1*R*A_T #T1
g3 = 0
# f4 = b_C*A_C - k1*R*A_T - k2*R*A_C - d_R*R #R
f4 = b_C*A_C - d_R*R #R
g4 = 0
f5 = d3*A_T - d2*C2 - a3*C2*T1 #C2
g5 = a2*C1*I1 
f6 = b_A*A_T - d_M*M_S #M_S
g6 = 0
f7 = k_tl*M_S - d_S*S - k_r*S #S
g7 = 0
f8 = k_r*S + d4*C_C - d_Ss*Ss# Ss
g8 =  - a4*Ss*I2 
# f9 = d4*C_C  #I_2
# g9 = - a4*Ss*I2
f9 = d5*A_C - d4*C_C - a5*C_C*T2 #C_C
g9 = a4*Ss*I2 
f10 = a5*C_C*T2 - d5*A_C - k2*R*A_C #A_C
g10 = 0
f11 = b_C*A_C - d_M*M_G #M_G
g11 = 0
f12 = k_G*M_G - d_G*G - k_r2*G #G
g12 = 0
f13 = k_r2*G - d_Gs*Gs #Gs
g13 = 0
f14 = d5*A_C + k2*R*A_C - a5*C_C*T2#T2
g14 = 0
# f16 = d1*C1 + d2*C2 #I1
# g16 = - a1*P1*I1 - a2*C1*I1
f = [f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14]
g = [g0,g1,g2,g3,g4,g5,g6,g7,g8,g9,g10,g11,g12,g13,g14]
# parameter values
params_values = [100, 10, 4, 10, 0.25, 2, 10, 0.5, 1, 1, 1000, 1000, 1000]
sys = System(x, f, params = params, g = g, u = u,  params_values = params_values, C = C, x_init = x_init)

In [14]:
from auto_reduce.utils import reduce
timepoints_ssm = np.linspace(0,20,10)
timepoints_ode = np.linspace(0,20,10)
sys_reduce = reduce(sys, timepoints_ode, timepoints_ssm)
sys_reduce.nstates_tol = 7

In [15]:
# P (0), A_T (1), C1 (2), T1 (3), R (4), C2 (5), M_S (6), S (7), Ss (8),
# C_C (9), A_C (10), M_G (11), G (12), Gs (13), T2 (14)
conserved_quantities = [A_T + T1 - T1_tot, A_C + T2 - T2_tot]
states_to_eliminate = [T1,T2]
f_cons = sys_reduce.set_conservation_laws(conserved_quantities, states_to_eliminate)

In [65]:
#  P1 (0), C1 (1), T1 (2), R (3), C2 (4), M_S (5), S (6), Ss (7), 
# , C_C (8), M_G (9), G (10), Gs (11), T2 (12)

# We want to retain R, M_S, C_C, M_G, G, Gs
reduced_sys, fast_ss = sys_reduce.solve_timescale_separation_with_input([R, M_S, A_T, C_C, Ss, M_G, G, Gs])


attempting : [3, 5, 1, 8, 7, 10, 11, 12]


In [91]:
reduced_sys.f[0]

-A_T*R*k1 + A_T*a3*d3*(-A_T + T1_tot)/(-A_T*a3 + T1_tot*a3 + d2) - A_T*d3

In [79]:
reduced_sys.x

[A_T, R, M_S, Ss, C_C, M_G, G, Gs]

In [8]:
f_cons

[C1*d1,
 -A_T*R*k1 - A_T*d3 + C2*a3*(-A_T + T1_tot),
 -C1*d1 + C2*d2,
 -A_C*R*k2 + A_C*b_C - A_T*R*k1 - R*d_R,
 A_T*d3 - C2*a3*(-A_T + T1_tot) - C2*d2,
 A_T*b_A - M_S*d_M,
 M_S*k_tl - S*d_S - S*k_r,
 C_C*d4 + S*k_r - Ss*d_Ss,
 A_C*d5 - C_C*a5*(-A_C + T2_tot) - C_C*d4,
 -A_C*R*k2 - A_C*d5 + C_C*a5*(-A_C + T2_tot),
 A_C*b_C - M_G*d_M,
 -G*d_G - G*k_r2 + M_G*k_G,
 G*k_r2 - Gs*d_Gs]

In [30]:
sys_reduce.x

[P1, A_T, C1, R, C2, M_S, S, Ss, C_C, A_C, M_G, G, Gs]

In [32]:
from sympy import latex
red_ff = reduced_sys.f
for k in range(len(red_ff)):
    print(latex(red_ff[k]))

- R d_{R} + \frac{b_{C} \left(C_{C} T_{2 tot} a_{5} + I_{2} Ss a_{4}\right)}{C_{C} a_{5} + R k_{2} + d_{5}}
- M_{S} d_{M} + \frac{T_{1 tot} a_{3} b_{A} \left(- \frac{R T_{1 tot} k_{1}}{d_{2}} - \frac{R k_{1}}{a_{3}} - \frac{d_{3}}{a_{3}}\right)}{R k_{1} + a_{3} \left(- \frac{R T_{1 tot} k_{1}}{d_{2}} - \frac{R k_{1}}{a_{3}} - \frac{d_{3}}{a_{3}}\right) + d_{3}} + d_{2} \left(- \frac{R T_{1 tot} k_{1}}{d_{2}} - \frac{R k_{1}}{a_{3}} - \frac{d_{3}}{a_{3}}\right)
C_{C} d_{4} + \frac{M_{S} k_{r} k_{tl}}{d_{S} + k_{r}} - Ss d_{Ss}
- C_{C} a_{5} \left(T_{2 tot} - \frac{C_{C} T_{2 tot} a_{5} + I_{2} Ss a_{4}}{C_{C} a_{5} + R k_{2} + d_{5}}\right) - C_{C} d_{4} - I_{2} Ss a_{4} + \frac{d_{5} \left(C_{C} T_{2 tot} a_{5} + I_{2} Ss a_{4}\right)}{C_{C} a_{5} + R k_{2} + d_{5}}
- M_{G} d_{M} + \frac{b_{C} \left(C_{C} T_{2 tot} a_{5} + I_{2} Ss a_{4}\right)}{C_{C} a_{5} + R k_{2} + d_{5}}
- G d_{G} - G k_{r2} + M_{G} k_{G}
G k_{r2} - Gs d_{Gs}


In [39]:
latex(reduced_sys.f[4])

'- M_{G} d_{M} + \\frac{b_{C} \\left(C_{C} T_{2 tot} a_{5} + I_{2} Ss a_{4}\\right)}{C_{C} a_{5} + R k_{2} + d_{5}}'

In [51]:
reduced_sys.f[0]

-R*d_R + b_C*(C_C*T2_tot*a5 + I2*Ss*a4)/(C_C*a5 + R*k2 + d5)

In [41]:
reduced_sys.x

[R, M_S, Ss, C_C, M_G, G, Gs]